In [3]:
import ee
import pandas as pd
import requests
import os

# Authenticate and initialize the Earth Engine API.
ee.Authenticate()
ee.Initialize()

# Read the metadata CSV file without headers (assume no header)
metadata = pd.read_csv('metadata.csv', header=None)

# Path to high PSNR images folder
high_psnr_hr_folder = 'high_psnr_hr'

# Function to find metadata based on image prefix using column indices
def get_metadata_by_prefix(prefix):
    # Assume the first column (index 0) contains the Landcover prefix
    rows = metadata[metadata.iloc[:, 0] == prefix]
    if rows.empty:
        raise ValueError(f"No metadata found for prefix: {prefix}")
    return rows.iloc[0]  # Get the first matching row

# Function to download a single Dynamic World image
def download_dw_image(bounds, date_start, date_end, output_path):
    try:
        # Convert dates to Earth Engine Date
        ee_date_start = ee.Date(date_start)
        ee_date_end = ee.Date(date_end)

        # Create a bounding box for the image
        ee_bounds = ee.Geometry.Rectangle(bounds)

        # Filter the Dynamic World collection by date and bounds
        dw_image = (ee.ImageCollection('GOOGLE/DYNAMICWORLD/V1')
                    .filterBounds(ee_bounds)
                    .filterDate(ee_date_start, ee_date_end)
                    .first())

        if not dw_image:
            raise ValueError("No Dynamic World image found for the given bounds and date range.")
        
        # Select the 'label' band (land cover classes)
        label_band = dw_image.select('label')

        # Generate a download URL for the image in GeoTIFF format
        url = label_band.getDownloadURL({
            'scale': 10,
            'region': ee_bounds,
            'format': 'GeoTIFF'
        })

        # Download the image and save it to the local file system
        response = requests.get(url, stream=True)
        if response.status_code == 200:
            with open(output_path, 'wb') as handler:
                for chunk in response.iter_content(1024):
                    handler.write(chunk)
            print(f"Downloaded image to {output_path}")
            
            # Check file size
            file_size = os.path.getsize(output_path)
            print(f"File size: {file_size} bytes")
        else:
            raise ValueError(f"Failed to download image, status code: {response.status_code}")

    except Exception as e:
        print(f"Error downloading Dynamic World image: {e}")

# Example usage
# Get a list of all high PSNR image prefixes
prefixes = [f.split('_')[0] for f in os.listdir(high_psnr_hr_folder) if f.endswith('_rgb.png')]

# Select the first prefix for testing
test_prefix = prefixes[0]

# Get the corresponding metadata for this prefix
meta = get_metadata_by_prefix(test_prefix)

# Ensure date_start is not after date_end
date_start = meta[3]  # Assuming lowres_date is in the 4th column (index 3)
date_end = meta[4]    # Assuming highres_date is in the 5th column (index 4)

# Swap the dates if necessary
if pd.to_datetime(date_start) > pd.to_datetime(date_end):
    print(f"Swapping dates for {test_prefix} because lowres_date is after highres_date.")
    date_start, date_end = date_end, date_start

# Download the Dynamic World image for this prefix
bounds = eval(meta[2])  # Assuming bounds is in the 3rd column (index 2)
output_path = f"{test_prefix}_dynamic_world_label.tif"

# Run the download
download_dw_image(bounds, date_start, date_end, output_path)


Downloaded image to Landcover-10236_dynamic_world_label.tif
File size: 2327 bytes


In [7]:
import rasterio
import numpy as np

# Specify the path to the downloaded image
tiff_output_path = "Landcover-10236_dynamic_world_label.tif"

# Open the image using rasterio
with rasterio.open(tiff_output_path) as src:
    # Get the number of bands in the image
    num_bands = src.count
    print(f"Number of bands: {num_bands}")
    
    # Retrieve the image metadata
    meta = src.meta
    print(f"Metadata: {meta}")
    
    # Get details for each band
    for i in range(1, num_bands + 1):
        band = src.read(i)
        print(f"Band {i}: min={band.min()}, max={band.max()}, mean={band.mean()}")

    # Check the band names (if available)
    band_names = src.descriptions
    print(f"Band names: {band_names}")

    # Check the Coordinate Reference System (CRS) information
    crs = src.crs
    print(f"Coordinate Reference System: {crs}")


Number of bands: 1
Metadata: {'driver': 'GTiff', 'dtype': 'uint8', 'nodata': 0.0, 'width': 164, 'height': 164, 'count': 1, 'crs': CRS.from_epsg(32634), 'transform': Affine(10.0, 0.0, 684720.0,
       0.0, 10.0, 5459900.0)}
Band 1: min=0, max=8, mean=3.376041046995836
Band names: (None,)
Coordinate Reference System: EPSG:32634


In [8]:
import rasterio

# Specify the path to the downloaded image
tiff_output_path = "Landcover-10236_dynamic_world_label.tif"

# Open the image using rasterio
with rasterio.open(tiff_output_path) as src:
    # Read the first (and only) band
    band = src.read(1)
    
    # Print the shape of the band to understand the dimensions
    print(f"Band shape: {band.shape}")
    
    # Option 1: Print all pixel values (if the image is small)
    print("All pixel values:")
    print(band)
    
    # Option 2: If the image is large, print pixel values in a specific region
    print("Pixel values in the top-left corner (10x10 region):")
    print(band[:10, :10])
    
    # Option 3: Print unique values in the band to see distinct land cover types
    unique_values = set(band.flatten())
    print(f"Unique pixel values (land cover types): {unique_values}")

    # Option 4: Print the count of each unique value (land cover type)
    unique, counts = np.unique(band, return_counts=True)
    print("Land cover type counts:")
    for value, count in zip(unique, counts):
        print(f"Value: {value}, Count: {count}")


Band shape: (164, 164)
All pixel values:
[[1 1 1 ... 1 1 1]
 [1 1 1 ... 1 1 1]
 [1 1 1 ... 1 1 1]
 ...
 [6 6 6 ... 5 5 5]
 [6 6 6 ... 5 5 5]
 [6 6 6 ... 1 1 1]]
Pixel values in the top-left corner (10x10 region):
[[1 1 1 1 1 1 1 1 1 1]
 [1 1 1 1 1 1 1 1 1 1]
 [1 1 1 1 1 1 1 1 1 1]
 [1 1 1 1 1 1 1 1 1 1]
 [1 1 1 1 1 1 1 1 1 1]
 [1 1 1 1 1 1 1 1 1 1]
 [1 1 1 1 1 1 1 1 1 1]
 [1 1 1 1 1 1 1 1 1 1]
 [1 1 1 1 1 1 1 1 1 1]
 [1 1 1 1 1 1 1 1 1 1]]
Unique pixel values (land cover types): {0, 1, 2, 4, 5, 6, 7, 8}
Land cover type counts:
Value: 0, Count: 68
Value: 1, Count: 11915
Value: 2, Count: 1437
Value: 4, Count: 778
Value: 5, Count: 4031
Value: 6, Count: 8290
Value: 7, Count: 10
Value: 8, Count: 367


In [9]:
import rasterio

# Specify the path to the corresponding LR image in high_psnr_lr folder
lr_image_path = "high_psnr_lr/Landcover-10236-L2A_data.tiff"

# Open the LR image using rasterio
with rasterio.open(lr_image_path) as src:
    # Get the dimensions (height and width) of the LR image
    width = src.width
    height = src.height
    
    print(f"LR Image Dimensions: width={width}, height={height}")
    
    # Optionally, you can print more details
    meta = src.meta
    print(f"Metadata: {meta}")


LR Image Dimensions: width=153, height=163
Metadata: {'driver': 'GTiff', 'dtype': 'float32', 'nodata': 0.0, 'width': 153, 'height': 163, 'count': 12, 'crs': CRS.from_epsg(4326), 'transform': Affine(0.00014200475107633206, 0.0, 23.539831607943892,
       0.0, -8.72206035367291e-05, 49.27816352448992)}


In [10]:
import ee
import pandas as pd
import requests
import os

# Authenticate and initialize the Earth Engine API.
ee.Authenticate()
ee.Initialize()

# Read the metadata CSV file without headers (assume no header)
metadata = pd.read_csv('metadata.csv', header=None)

# Path to high PSNR images folder
high_psnr_hr_folder = 'high_psnr_hr'

# Function to find metadata based on image prefix using column indices
def get_metadata_by_prefix(prefix):
    # Assume the first column (index 0) contains the Landcover prefix
    rows = metadata[metadata.iloc[:, 0] == prefix]
    if rows.empty:
        raise ValueError(f"No metadata found for prefix: {prefix}")
    return rows.iloc[0]  # Get the first matching row

# Function to download a single Dynamic World image
def download_dw_image(bounds, date_start, date_end, output_path):
    try:
        # Convert dates to Earth Engine Date
        ee_date_start = ee.Date(date_start)
        ee_date_end = ee.Date(date_end)

        # Create a bounding box for the image
        ee_bounds = ee.Geometry.Rectangle(bounds)

        # Filter the Dynamic World collection by date and bounds
        dw_image = (ee.ImageCollection('GOOGLE/DYNAMICWORLD/V1')
                    .filterBounds(ee_bounds)
                    .filterDate(ee_date_start, ee_date_end)
                    .first())

        if not dw_image:
            raise ValueError("No Dynamic World image found for the given bounds and date range.")
        
        # Select the 'label' band (land cover classes)
        label_band = dw_image.select('label')

        # Generate a download URL for the image in GeoTIFF format
        url = label_band.getDownloadURL({
            'scale': 10,
            'region': ee_bounds,
            'format': 'GeoTIFF'
        })

        # Download the image and save it to the local file system
        response = requests.get(url, stream=True)
        if response.status_code == 200:
            with open(output_path, 'wb') as handler:
                for chunk in response.iter_content(1024):
                    handler.write(chunk)
            print(f"Downloaded image to {output_path}")
            
            # Check file size
            file_size = os.path.getsize(output_path)
            print(f"File size: {file_size} bytes")
        else:
            raise ValueError(f"Failed to download image, status code: {response.status_code}")

    except Exception as e:
        print(f"Error downloading Dynamic World image: {e}")

# Create the output directory if it doesn't exist
output_folder = 'lc_images'
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

# Process each file in the high_psnr_hr folder
for file_name in os.listdir(high_psnr_hr_folder):
    if file_name.endswith('_rgb.png'):
        # Extract the prefix from the file name
        prefix = file_name.split('_')[0]

        # Get the corresponding metadata for this prefix
        try:
            meta = get_metadata_by_prefix(prefix)
        except ValueError as e:
            print(e)
            continue

        # Ensure date_start is not after date_end
        date_start = meta[3]  # Assuming lowres_date is in the 4th column (index 3)
        date_end = meta[4]    # Assuming highres_date is in the 5th column (index 4)

        # Swap the dates if necessary
        if pd.to_datetime(date_start) > pd.to_datetime(date_end):
            print(f"Swapping dates for {prefix} because lowres_date is after highres_date.")
            date_start, date_end = date_end, date_start

        # Define the bounds and output path
        bounds = eval(meta[2])  # Assuming bounds is in the 3rd column (index 2)
        output_path = os.path.join(output_folder, f"{prefix}_dynamic_world_label.tif")

        # Download the Dynamic World image for this prefix
        download_dw_image(bounds, date_start, date_end, output_path)

print("Batch download complete.")


Downloaded image to lc_images/Landcover-10236_dynamic_world_label.tif
File size: 2327 bytes
Downloaded image to lc_images/Landcover-103651_dynamic_world_label.tif
File size: 1086 bytes
Error downloading Dynamic World image: Collection.first: Empty date ranges not supported for the current operation.
Swapping dates for Landcover-1058565 because lowres_date is after highres_date.
Error downloading Dynamic World image: Image.select: Parameter 'input' is required.
Swapping dates for Landcover-1060086 because lowres_date is after highres_date.
Downloaded image to lc_images/Landcover-1060086_dynamic_world_label.tif
File size: 961 bytes
Error downloading Dynamic World image: Collection.first: Empty date ranges not supported for the current operation.
Downloaded image to lc_images/Landcover-1062315_dynamic_world_label.tif
File size: 611 bytes
Downloaded image to lc_images/Landcover-1089999_dynamic_world_label.tif
File size: 480 bytes
Error downloading Dynamic World image: Collection.first: Emp

In [11]:
import os

# Specify the folder containing the downloaded images
lc_images_folder = 'lc_images'

# List all files in the folder
files = os.listdir(lc_images_folder)

# Filter the list to only include files with .tif extension
tif_files = [f for f in files if f.endswith('.tif')]

# Count the number of .tif files
num_images = len(tif_files)

print(f"There are {num_images} images in the '{lc_images_folder}' folder.")


There are 842 images in the 'lc_images' folder.


In [14]:
import os
import shutil

# Define the folder paths
lc_images_folder = 'lc_images'
high_psnr_hr_folder = 'high_psnr_hr'
high_psnr_lr_folder = 'high_psnr_lr'

# Define the output folders for the selected images
hr_output_folder = 'hr_images'
lr_output_folder = 'lr_images'

# Create the output folders if they don't exist
os.makedirs(hr_output_folder, exist_ok=True)
os.makedirs(lr_output_folder, exist_ok=True)

# Get the list of LC image filenames (without extensions)
lc_filenames = [os.path.splitext(f)[0] for f in os.listdir(lc_images_folder) if f.endswith('.tif')]

# Iterate over the LC filenames and find corresponding HR and LR images
for lc_filename in lc_filenames:
    # Extract the prefix (e.g., 'Landcover-XXXX') from the LC filename
    prefix = lc_filename.split('_')[0]
    
    # Build the expected HR and LR filenames
    hr_filename = f"{prefix}_rgb.png"
    lr_filename = f"{prefix}-L2A_data.tiff"
    
    # Check if the corresponding HR image exists in the high_psnr_hr folder
    hr_image_path = os.path.join(high_psnr_hr_folder, hr_filename)
    if os.path.exists(hr_image_path):
        # Copy the HR image to the hr_images folder
        shutil.copy(hr_image_path, os.path.join(hr_output_folder, hr_filename))
    
    # Check if the corresponding LR image exists in the high_psnr_lr folder
    lr_image_path = os.path.join(high_psnr_lr_folder, lr_filename)
    if os.path.exists(lr_image_path):
        # Copy the LR image to the lr_images folder
        shutil.copy(lr_image_path, os.path.join(lr_output_folder, lr_filename))

# Count the number of images in the hr_images and lr_images folders
num_hr_images = len(os.listdir(hr_output_folder))
num_lr_images = len(os.listdir(lr_output_folder))

print(f"There are {num_hr_images} images in the 'hr_images' folder.")
print(f"There are {num_lr_images} images in the 'lr_images' folder.")


There are 842 images in the 'hr_images' folder.
There are 842 images in the 'lr_images' folder.


In [23]:
import rasterio

# Specify the path to the downloaded image
tiff_output_path = "lc_images/Landcover-1060086_dynamic_world_label.tif"

# Open the image using rasterio
with rasterio.open(tiff_output_path) as src:
    # Read the first (and only) band
    band = src.read(1)
    
    # Print the shape of the band to understand the dimensions
    print(f"Band shape: {band.shape}")
    
    # Option 1: Print all pixel values (if the image is small)
    print("All pixel values:")
    print(band)
    
    # Option 2: If the image is large, print pixel values in a specific region
    print("Pixel values in the top-left corner (10x10 region):")
    print(band[:10, :10])
    
    # Option 3: Print unique values in the band to see distinct land cover types
    unique_values = set(band.flatten())
    print(f"Unique pixel values (land cover types): {unique_values}")

    # Option 4: Print the count of each unique value (land cover type)
    unique, counts = np.unique(band, return_counts=True)
    print("Land cover type counts:")
    for value, count in zip(unique, counts):
        print(f"Value: {value}, Count: {count}")


Band shape: (167, 168)
All pixel values:
[[0 0 0 ... 1 1 1]
 [0 0 0 ... 1 1 1]
 [0 0 0 ... 1 1 1]
 ...
 [1 1 1 ... 1 1 1]
 [1 1 1 ... 1 1 1]
 [1 1 1 ... 1 1 1]]
Pixel values in the top-left corner (10x10 region):
[[0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]]
Unique pixel values (land cover types): {0, 1, 2, 5, 7}
Land cover type counts:
Value: 0, Count: 12276
Value: 1, Count: 15208
Value: 2, Count: 120
Value: 5, Count: 445
Value: 7, Count: 7


In [24]:
import rasterio

# Specify the path to the corresponding LR image in high_psnr_lr folder
lr_image_path = "lr_images/Landcover-1060086-L2A_data.tiff"

# Open the LR image using rasterio
with rasterio.open(lr_image_path) as src:
    # Get the dimensions (height and width) of the LR image
    width = src.width
    height = src.height
    
    print(f"LR Image Dimensions: width={width}, height={height}")
    
    # Optionally, you can print more details
    meta = src.meta
    print(f"Metadata: {meta}")


LR Image Dimensions: width=164, height=152
Metadata: {'driver': 'GTiff', 'dtype': 'float32', 'nodata': 0.0, 'width': 164, 'height': 152, 'count': 12, 'crs': CRS.from_epsg(4326), 'transform': Affine(0.00018491086133417897, 0.0, 36.50710720486381,
       0.0, -9.333704435280541e-05, 62.15786553230777)}


In [26]:
import os
import rasterio
import numpy as np

# Folder paths
lr_images_folder = 'lr_images'
lc_images_folder = 'lc_images'
resized_lc_images_folder = 'resized_lc_images'

# Create folder for saving cropped LC images
if not os.path.exists(resized_lc_images_folder):
    os.makedirs(resized_lc_images_folder)

# Loop through images in lr_images folder
for lr_image_filename in os.listdir(lr_images_folder):
    if lr_image_filename.endswith('.tiff') or lr_image_filename.endswith('.tif'):
        lr_image_path = os.path.join(lr_images_folder, lr_image_filename)
        
        # Extract prefix to match with LC images
        prefix = lr_image_filename.split('-')[1]  # Extract the prefix from the image filename
        lc_image_filename = f"Landcover-{prefix}_dynamic_world_label.tif"
        lc_image_path = os.path.join(lc_images_folder, lc_image_filename)
        
        if not os.path.exists(lc_image_path):
            print(f"LC image not found for {lr_image_filename}")
            continue
        
        # Read the dimensions of the LR image
        with rasterio.open(lr_image_path) as lr_src:
            lr_width = lr_src.width
            lr_height = lr_src.height
        
        # Read the LC image and crop it
        with rasterio.open(lc_image_path) as lc_src:
            lc_image = lc_src.read(1)  # Read the first band of the LC image
            lc_height, lc_width = lc_image.shape
            
            # Calculate the start point for center cropping
            center_x = lc_width // 2
            center_y = lc_height // 2
            start_x = center_x - lr_width // 2
            start_y = center_y - lr_height // 2
            
            # Ensure the crop does not go out of bounds
            start_x = max(start_x, 0)
            start_y = max(start_y, 0)
            end_x = min(start_x + lr_width, lc_width)
            end_y = min(start_y + lr_height, lc_height)
            
            # Crop the LC image
            cropped_lc_image = lc_image[start_y:end_y, start_x:end_x]
            
            # Check if the cropped dimensions match the LR image dimensions
            if cropped_lc_image.shape[0] != lr_height or cropped_lc_image.shape[1] != lr_width:
                print(f"Cropping dimensions do not match for {lr_image_filename}")
                continue
            
            # Save the cropped LC image with the original name
            cropped_lc_image_path = os.path.join(resized_lc_images_folder, lc_image_filename)
            with rasterio.open(cropped_lc_image_path, 'w', driver='GTiff', height=cropped_lc_image.shape[0], width=cropped_lc_image.shape[1], count=1, dtype=cropped_lc_image.dtype) as dst:
                dst.write(cropped_lc_image, 1)

            print(f"Saved resized LC image for {lr_image_filename} as {cropped_lc_image_path}")
            

Saved resized LC image for Landcover-10236-L2A_data.tiff as resized_lc_images/Landcover-10236_dynamic_world_label.tif
Saved resized LC image for Landcover-103651-L2A_data.tiff as resized_lc_images/Landcover-103651_dynamic_world_label.tif
Saved resized LC image for Landcover-1060086-L2A_data.tiff as resized_lc_images/Landcover-1060086_dynamic_world_label.tif
Saved resized LC image for Landcover-1062315-L2A_data.tiff as resized_lc_images/Landcover-1062315_dynamic_world_label.tif
Saved resized LC image for Landcover-1089999-L2A_data.tiff as resized_lc_images/Landcover-1089999_dynamic_world_label.tif
Saved resized LC image for Landcover-126967-L2A_data.tiff as resized_lc_images/Landcover-126967_dynamic_world_label.tif
Saved resized LC image for Landcover-127471-L2A_data.tiff as resized_lc_images/Landcover-127471_dynamic_world_label.tif
Saved resized LC image for Landcover-129921-L2A_data.tiff as resized_lc_images/Landcover-129921_dynamic_world_label.tif
Saved resized LC image for Landcover

In [28]:
import rasterio

# Specify the path to the downloaded image
tiff_output_path = "resized_lc_images/Landcover-10236_dynamic_world_label.tif"

# Open the image using rasterio
with rasterio.open(tiff_output_path) as src:
    # Read the first (and only) band
    band = src.read(1)
    
    # Print the shape of the band to understand the dimensions
    print(f"Band shape: {band.shape}")
    
    # Option 1: Print all pixel values (if the image is small)
    print("All pixel values:")
    print(band)
    
    # Option 2: If the image is large, print pixel values in a specific region
    print("Pixel values in the top-left corner (10x10 region):")
    print(band[:10, :10])
    
    # Option 3: Print unique values in the band to see distinct land cover types
    unique_values = set(band.flatten())
    print(f"Unique pixel values (land cover types): {unique_values}")

    # Option 4: Print the count of each unique value (land cover type)
    unique, counts = np.unique(band, return_counts=True)
    print("Land cover type counts:")
    for value, count in zip(unique, counts):
        print(f"Value: {value}, Count: {count}")


Band shape: (163, 153)
All pixel values:
[[1 1 1 ... 1 1 1]
 [1 1 1 ... 1 1 1]
 [1 1 1 ... 1 1 1]
 ...
 [6 6 6 ... 5 1 1]
 [6 6 6 ... 1 1 1]
 [6 6 6 ... 1 1 1]]
Pixel values in the top-left corner (10x10 region):
[[1 1 1 1 1 1 1 1 1 1]
 [1 1 1 1 1 1 1 1 1 1]
 [1 1 1 1 1 1 1 1 1 1]
 [1 1 1 1 1 1 1 1 1 1]
 [1 1 1 1 1 1 1 1 1 1]
 [1 1 1 1 1 1 1 1 1 1]
 [1 1 1 1 1 1 1 1 1 1]
 [1 1 1 1 1 1 1 1 1 1]
 [1 1 1 1 1 1 1 1 1 1]
 [1 1 1 1 1 1 1 1 1 1]]
Unique pixel values (land cover types): {0, 1, 2, 4, 5, 6, 7, 8}
Land cover type counts:
Value: 0, Count: 68
Value: 1, Count: 10572
Value: 2, Count: 1420
Value: 4, Count: 764
Value: 5, Count: 3775
Value: 6, Count: 8005
Value: 7, Count: 10
Value: 8, Count: 325
